In [293]:
!pip install langchain langchain_community langchainhub openai PyPDF2 langchain_qdrant tiktoken langgraph langchain_openai

You should consider upgrading via the '/Users/felipeescobar/Documents/llm_fastapi/venv/bin/python3 -m pip install --upgrade pip' command.


In [390]:
import uuid

from langchain.chat_models.openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.retrievers.multi_query import MultiQueryRetriever
import PyPDF2

In [183]:
with open("protocolo-clinico-e-diretrizes-terapeuticas-do-tabagismo.pdf", "rb") as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            text += page.extract_text()

In [1]:
api_key = ""

In [186]:
splitter = RecursiveCharacterTextSplitter(chunk_size=4000, chunk_overlap=0)

In [272]:
docs = splitter.create_documents([text])


In [273]:
len(docs)

34

In [192]:
chain = (
    {"doc": lambda x: x.page_content}
    | ChatPromptTemplate.from_template("Summarize the following document:\n\n{doc}")
    | ChatOpenAI(max_retries=0, api_key =api_key)
    | StrOutputParser()
)

In [193]:
summaries = chain.batch(docs, {"max_concurrency": 5})

In [194]:
summaries

['A portaria conjunta nº 10, de 16 de abril de 2020, do Ministério da Saúde, aprova o Protocolo Clínico e Diretrizes Terapêuticas do Tabagismo. O documento considera a necessidade de atualização de parâmetros sobre o tabagismo no Brasil e diretrizes nacionais para diagnóstico, tratamento e acompanhamento dos indivíduos com essa doença. Estabelece que o protocolo seja utilizado pelas Secretarias de Saúde dos Estados, do Distrito Federal e dos Municípios na regulação do acesso assistencial. Também determina que os gestores definam os serviços referenciais e estabeleçam os fluxos para o atendimento dos pacientes com tabagismo. A portaria entra em vigor na data de sua publicação e revoga a Portaria nº 761/SAS/MS, de 21 de junho de 2016. O tabagismo é considerado uma doença epidêmica decorrente da dependência à nicotina, sendo fator causal de várias doenças incapacitantes e fatais, como câncer, doenças cardiovasculares e respiratórias crônicas.',
 'O documento descreve as medidas adotadas p

In [397]:
from langchain_qdrant import Qdrant
from langchain.embeddings import OpenAIEmbeddings
from langchain.storage import InMemoryByteStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import models

qdrant_client = QdrantClient(
    api_key="woXKPNeAD8uq1fVpLLPrDMX5AM4xyjAnLDPzszQuzZKFn",
    https=False,
)

try:
    qdrant_client.get_collection(collection_name="summaries-tgs")
except Exception as e:
    qdrant_client.create_collection(collection_name="summaries-tgs",vectors_config=models.VectorParams(
                size=1536, distance=models.Distance.COSINE
            ))
    
embedding_function=OpenAIEmbeddings(api_key =api_key)

vectorstore = Qdrant(client=qdrant_client ,collection_name="summaries-tgs",embeddings=embedding_function)

# The storage layer for the parent documents
store = InMemoryByteStore()

id_key = "doc_id"

retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    byte_store=store,
    id_key=id_key,
)

# retriever = vectorstore.as_retriever(
#     search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.5}
# )
# retriever = vectorstore.as_retriever(
#             search_type="mmr",
#             search_kwargs={"k": 5, "lambda_mult": 0.25, "fetch_k": 50},
#         )
doc_ids = [str(uuid.uuid4()) for _ in docs]

/Users/felipeescobar/Documents/llm_fastapi/venv/lib/python3.10/site-packages/qdrant_client/qdrant_remote.py:130: UserWarning: Api key is used with an insecure connection.
  warnings.warn("Api key is used with an insecure connection.")


In [275]:
summary_docs = [
    Document(page_content=s, metadata={id_key: doc_ids[i]})
    for i, s in enumerate(summaries)
]

In [276]:
retriever.vectorstore.add_documents(summary_docs)

retriever.docstore.mset(list(zip(doc_ids, docs)))

In [316]:
#docs = retriever.get_relevant_documents(query="qual abordagem breve a todo fumante?")